In [44]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot
from IPython.display import Markdown as md



In [45]:
init_notebook_mode(connected=True)


In [46]:
def coefficient_of_determination(y, p_y):
    assert (len(y) == len(p_y)), "Vectors must be same length"
    y_m = np.mean(y)
    s_s_tot = np.sum((y - y_m)**2)
    residuals = np.diff(np.array([y, p_y]), axis=0)
    s_s_res = np.sum(residuals**2)
    return 1 - s_s_res / s_s_tot


def compute_cost_function(theta, x, y):
    x_arr = np.asarray(x) if type(x) is list else x
    m = x_arr.shape[0]
    ones = np.ones((m, 1))
    xa = np.hstack((ones, x))
    approx_value = np.sum(theta * xa, axis=1)
    dif = approx_value - y
    return sum(dif ** 2) / (2 * m)


def compute_gradient(theta, x, y):
    x_arr = np.asarray(x) if type(x) is list else x
    m = x_arr.shape[0]
    ones = np.ones((m, 1))
    xa = np.hstack((ones, x))
    approx_value = np.sum(theta * xa, axis=1)
    dif = approx_value - y
    grad0 = np.asarray([sum(dif) / m])
    gradn = np.dot(np.asarray(dif), x) / m
    grad = np.concatenate((grad0, gradn), axis=0)
    return grad


def propose_theta(data):
    return np.ones(data.shape[1])


def update_theta(theta, gradient, alpha):
    return theta - alpha * gradient


def calc_theta_for_batch(batch_data, min_theta, alpha):
    batch_n = batch_data.shape[1] - 1
    batch_x = batch_data[:, range(batch_n)]
    batch_y = batch_data[:, batch_n]
    g = compute_gradient(min_theta, batch_x, batch_y)
    return update_theta(min_theta, g, alpha)


def vanilla_gradient_descent(data, start_theta=None, alpha=None, max_iter=None):

    if start_theta is None:
        start_theta = propose_theta(data)

    if alpha is None:
        alpha = 0.001

    if max_iter is None:
        max_iter = 40


    f_n = data.shape[1] - 1
    x = data[:, range(f_n)]
    y = data[:, f_n]
    iter_num = 0
    cur_theta = start_theta
    while iter_num < max_iter:
        gradient = compute_gradient(cur_theta, x, y)
        cur_theta = update_theta(cur_theta, gradient, alpha)
        iter_num += 1
    return cur_theta


def mini_batch_gradient_descent(data, start_theta=None, alpha=None, max_iter=None, batch_size=5):
    if start_theta is None:
        start_theta = propose_theta(data)

    if alpha is None:
        alpha = 0.001

    if max_iter is None:
        max_iter = 100

    min_theta = start_theta
    min_cost = float("inf")
    f_n = data.shape[1] - 1

    iteration_without_improvement = 0
    cost_history = []
    cur_alpha = alpha
    while iteration_without_improvement < max_iter:
        np.random.shuffle(data)
        batch_data = data[0:batch_size]
        cur_theta = calc_theta_for_batch(batch_data, min_theta, cur_alpha)
        x = data[:, range(f_n)]
        y = data[:, f_n]
        cur_cost = compute_cost_function(cur_theta, x, y)
        cost_history.append(cur_cost)
        if cur_cost < min_cost:
            min_cost = cur_cost
            min_theta = cur_theta
            iteration_without_improvement = 0
            cur_alpha *= 0.95
        else:
            iteration_without_improvement+=1

    return min_theta, min(cost_history)

In [47]:
df_adv = pd.read_csv('http://www-bcf.usc.edu/~gareth/ISL/Advertising.csv', index_col=0)
x = df_adv['TV']
y = df_adv['radio']
z = df_adv['sales']



In [48]:
df_adv.head()

,TV,radio,newspaper,sales
1,230.1,37.8,69.2,22.1
2,44.5,39.3,45.1,10.4
3,17.2,45.9,69.3,9.3
4,151.5,41.3,58.5,18.5
5,180.8,10.8,58.4,12.9


In [49]:
md("**Multiple Linear Regression Example** ")


**Multiple Linear Regression Example** 

In [50]:

md("__Sales = beta0 + beta1 * TV + beta2*Radio__")


__Sales = beta0 + beta1 * TV + beta2*Radio__

In [ ]:
r = np.column_stack((x,y,z))    
theta = vanilla_gradient_descent(r, start_theta=(1, 0, 0), alpha = 0.00005, max_iter=5500)

t1 = range(-150,450)
t2 = range(-30,80)
xv, yv = np.meshgrid(t1, t2)
xvf = xv.flatten()
yvf = yv.flatten()
theta0 = theta[0]
theta1 = theta[1]
theta2 = theta[2]
pred=theta0+theta1*xvf+theta2*yvf 
print(theta0, theta1, theta2)





1.0877558730806958 0.051460796284317184 0.21789425292685724


In [80]:
data_trace = go.Scatter3d(
    x=x,
    y=y,
    z=z,
    mode='markers',
    name='TV + Radio'
)


In [81]:
reg_trace=go.Mesh3d(x=xvf,y=yvf,z=pred,color='red',opacity=0.5, name='Predicted Sales')


In [82]:
rmse = np.sqrt(np.mean((theta0+theta1*x+theta2*y-z)**2))
md("__Root-Mean Square Error = %r__"%(np.around(rmse, decimals=2)))


__Root-Mean Square Error = 1.83__

In [83]:
pred_c = theta0+theta1*x+theta2*y
cd = coefficient_of_determination(y, pred_c)
md("__Coefficient of Determination = %r__"%(np.around(cd, decimals=3)))


__Coefficient of Determination = -0.09__

In [61]:
fig = go.Figure(data=[data_trace,reg_trace])
iplot(fig)

<div id="875066c6-bd5e-4c9b-91e4-0316ad78551e" style="height: 525px; width: 100%;" class="plotly-graph-div"></div><script type="text/javascript">require(["plotly"], function(Plotly) { window.PLOTLYENV=window.PLOTLYENV || {};window.PLOTLYENV.BASE_URL="https://plot.ly";Plotly.newPlot("875066c6-bd5e-4c9b-91e4-0316ad78551e", [{"mode": "markers", "name": "TV + Radio", "x": [230.1, 44.5, 17.2, 151.5, 180.8, 8.7, 57.5, 120.2, 8.6, 199.8, 66.1, 214.7, 23.8, 97.5, 204.1, 195.4, 67.8, 281.4, 69.2, 147.3, 218.4, 237.4, 13.2, 228.3, 62.3, 262.9, 142.9, 240.1, 248.8, 70.6, 292.9, 112.9, 97.2, 265.6, 95.7, 290.7, 266.9, 74.7, 43.1, 228.0, 202.5, 177.0, 293.6, 206.9, 25.1, 175.1, 89.7, 239.9, 227.2, 66.9, 199.8, 100.4, 216.4, 182.6, 262.7, 198.9, 7.3, 136.2, 210.8, 210.7, 53.5, 261.3, 239.3, 102.7, 131.1, 69.0, 31.5, 139.3, 237.4, 216.8, 199.1, 109.8, 26.8, 129.4, 213.4, 16.9, 27.5, 120.5, 5.4, 116.0, 76.4, 239.8, 75.3, 68.4, 213.5, 193.2, 76.3, 110.7, 88.3, 109.8, 134.3, 28.6, 217.7, 250.9, 107.4, 163.3, 197.6, 184.9, 289.7, 135.2, 222.4, 296.4, 280.2, 187.9, 238.2, 137.9, 25.0, 90.4, 13.1, 255.4, 225.8, 241.7, 175.7, 209.6, 78.2, 75.1, 139.2, 76.4, 125.7, 19.4, 141.3, 18.8, 224.0, 123.1, 229.5, 87.2, 7.8, 80.2, 220.3, 59.6, 0.7, 265.2, 8.4, 219.8, 36.9, 48.3, 25.6, 273.7, 43.0, 184.9, 73.4, 193.7, 220.5, 104.6, 96.2, 140.3, 240.1, 243.2, 38.0, 44.7, 280.7, 121.0, 197.6, 171.3, 187.8, 4.1, 93.9, 149.8, 11.7, 131.7, 172.5, 85.7, 188.4, 163.5, 117.2, 234.5, 17.9, 206.8, 215.4, 284.3, 50.0, 164.5, 19.6, 168.4, 222.4, 276.9, 248.4, 170.2, 276.7, 165.6, 156.6, 218.5, 56.2, 287.6, 253.8, 205.0, 139.5, 191.1, 286.0, 18.7, 39.5, 75.5, 17.2, 166.8, 149.7, 38.2, 94.2, 177.0, 283.6, 232.1], "y": [37.8, 39.3, 45.9, 41.3, 10.8, 48.9, 32.8, 19.6, 2.1, 2.6, 5.8, 24.0, 35.1, 7.6, 32.9, 47.7, 36.6, 39.6, 20.5, 23.9, 27.7, 5.1, 15.9, 16.9, 12.6, 3.5, 29.3, 16.7, 27.1, 16.0, 28.3, 17.4, 1.5, 20.0, 1.4, 4.1, 43.8, 49.4, 26.7, 37.7, 22.3, 33.4, 27.7, 8.4, 25.7, 22.5, 9.9, 41.5, 15.8, 11.7, 3.1, 9.6, 41.7, 46.2, 28.8, 49.4, 28.1, 19.2, 49.6, 29.5, 2.0, 42.7, 15.5, 29.6, 42.8, 9.3, 24.6, 14.5, 27.5, 43.9, 30.6, 14.3, 33.0, 5.7, 24.6, 43.7, 1.6, 28.5, 29.9, 7.7, 26.7, 4.1, 20.3, 44.5, 43.0, 18.4, 27.5, 40.6, 25.5, 47.8, 4.9, 1.5, 33.5, 36.5, 14.0, 31.6, 3.5, 21.0, 42.3, 41.7, 4.3, 36.3, 10.1, 17.2, 34.3, 46.4, 11.0, 0.3, 0.4, 26.9, 8.2, 38.0, 15.4, 20.6, 46.8, 35.0, 14.3, 0.8, 36.9, 16.0, 26.8, 21.7, 2.4, 34.6, 32.3, 11.8, 38.9, 0.0, 49.0, 12.0, 39.6, 2.9, 27.2, 33.5, 38.6, 47.0, 39.0, 28.9, 25.9, 43.9, 17.0, 35.4, 33.2, 5.7, 14.8, 1.9, 7.3, 49.0, 40.3, 25.8, 13.9, 8.4, 23.3, 39.7, 21.1, 11.6, 43.5, 1.3, 36.9, 18.4, 18.1, 35.8, 18.1, 36.8, 14.7, 3.4, 37.6, 5.2, 23.6, 10.6, 11.6, 20.9, 20.1, 7.1, 3.4, 48.9, 30.2, 7.8, 2.3, 10.0, 2.6, 5.4, 5.7, 43.0, 21.3, 45.1, 2.1, 28.7, 13.9, 12.1, 41.1, 10.8, 4.1, 42.0, 35.6, 3.7, 4.9, 9.3, 42.0, 8.6], "z": [22.1, 10.4, 9.3, 18.5, 12.9, 7.2, 11.8, 13.2, 4.8, 10.6, 8.6, 17.4, 9.2, 9.7, 19.0, 22.4, 12.5, 24.4, 11.3, 14.6, 18.0, 12.5, 5.6, 15.5, 9.7, 12.0, 15.0, 15.9, 18.9, 10.5, 21.4, 11.9, 9.6, 17.4, 9.5, 12.8, 25.4, 14.7, 10.1, 21.5, 16.6, 17.1, 20.7, 12.9, 8.5, 14.9, 10.6, 23.2, 14.8, 9.7, 11.4, 10.7, 22.6, 21.2, 20.2, 23.7, 5.5, 13.2, 23.8, 18.4, 8.1, 24.2, 15.7, 14.0, 18.0, 9.3, 9.5, 13.4, 18.9, 22.3, 18.3, 12.4, 8.8, 11.0, 17.0, 8.7, 6.9, 14.2, 5.3, 11.0, 11.8, 12.3, 11.3, 13.6, 21.7, 15.2, 12.0, 16.0, 12.9, 16.7, 11.2, 7.3, 19.4, 22.2, 11.5, 16.9, 11.7, 15.5, 25.4, 17.2, 11.7, 23.8, 14.8, 14.7, 20.7, 19.2, 7.2, 8.7, 5.3, 19.8, 13.4, 21.8, 14.1, 15.9, 14.6, 12.6, 12.2, 9.4, 15.9, 6.6, 15.5, 7.0, 11.6, 15.2, 19.7, 10.6, 6.6, 8.8, 24.7, 9.7, 1.6, 12.7, 5.7, 19.6, 10.8, 11.6, 9.5, 20.8, 9.6, 20.7, 10.9, 19.2, 20.1, 10.4, 11.4, 10.3, 13.2, 25.4, 10.9, 10.1, 16.1, 11.6, 16.6, 19.0, 15.6, 3.2, 15.3, 10.1, 7.3, 12.9, 14.4, 13.3, 14.9, 18.0, 11.9, 11.9, 8.0, 12.2, 17.1, 15.0, 8.4, 14.5, 7.6, 11.7, 11.5, 27.0, 20.2, 11.7, 11.8, 12.6, 10.5, 12.2, 8.7, 26.2, 17.6, 22.6, 10.3, 17.3, 15.9, 6.7, 10.8, 9.9, 5.9, 19.6, 17.3, 7.6, 9.7, 12.8,